In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
import glob
import scipy.stats
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
plt.style.use('ggplot')
import os
if not os.path.exists("images"):
    os.mkdir("images")
import kaleido 
import plotly.graph_objects as go
import math
from scipy import stats
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

### Convert some variables from numerical to categorical or vise versa
### Fill in null values appropriately 
### Feature engineering 

In [106]:
housing = pd.read_csv('Ames_HousePrice.csv', index_col=0)

# reset index so there is only one number associated with each house
housing = housing.reset_index(drop = True)

# deletes duplicate point 
housing = housing.drop_duplicates()

#Convert some numerical variables to categorical 
housing.MSSubClass = housing.MSSubClass.apply(lambda x: str(x))
housing.MoSold = housing.MoSold.apply(lambda x: str(x))
housing.YrSold = housing.YrSold.apply(lambda x: str(x))

# Alley
housing.Alley.replace({'Grvl':1, 'Pave':2}, inplace=True)

# Lot Shape
housing.LotShape.replace({'Reg':1, 'IR1':2, 'IR2':3, 'IR3':4}, inplace=True)

# Land Contour
housing.LandContour.replace({'Low':1, 'HLS':2, 'Bnk':3, 'Lvl':4}, inplace=True)

# Utilities
housing.Utilities.replace({'ELO':1, 'NoSeWa':2, 'NoSewr':3, 'AllPub':4}, inplace=True)

# Land Slope..idk if I'm going to inlude this 
# further investigation there is correlation ..severe slope = more expensiveaaa
housing.LandSlope.replace({'Sev':1, 'Mod':2, 'Gtl':3}, inplace=True)

# Exterior Quality
housing.ExterQual.replace({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}, inplace=True)

# Exterior Condition
housing.ExterCond.replace({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}, inplace=True)

# Basement Quality
housing.BsmtQual.replace({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}, inplace=True)

# Basement Condition
housing.BsmtCond.replace({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}, inplace=True)

# Basement Exposure
housing.BsmtExposure.replace({'No':1, 'Mn':2, 'Av':3, 'Gd':4}, inplace=True)

# Finished Basement 1 Rating
housing.BsmtFinType1.replace({'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6}, inplace=True)

# Finished Basement 2 Rating
housing.BsmtFinType2.replace({'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6}, inplace=True)

# Heating Quality and Condition
housing.HeatingQC.replace({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}, inplace=True)

# Kitchen Quality
housing.KitchenQual.replace({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}, inplace=True)

# Home functionality
housing.Functional.replace({'Sal':1, 'Sev':2, 'Maj2':3, 'Maj1':4, 'Mod':5, 'Min2':6, 'Min1':7, 'Typ':8}, inplace=True)

# Fireplace Quality
housing.FireplaceQu.replace({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}, inplace=True)

# Garage Finish
housing.GarageFinish.replace({'Unf':1, 'RFn':2, 'Fin':3}, inplace=True)

# Garage Quality
housing.GarageQual.replace({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}, inplace=True)

# Garage Condition
housing.GarageCond.replace({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}, inplace=True)

# Paved Driveway
housing.PavedDrive.replace({'N':1, 'P':2, 'Y':3}, inplace=True)

# Pool Quality
housing.PoolQC.replace({'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}, inplace=True)

# for the converted features fill in nulls with 0
converted_features = ['Alley','LotShape','LandContour','Utilities','LandSlope','ExterQual','ExterCond',
        'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','HeatingQC',
        'KitchenQual','Functional','FireplaceQu','GarageFinish','GarageQual',
        'GarageCond','PavedDrive','PoolQC']
housing[converted_features] = housing[converted_features].fillna(0)

# some categorical nulls will be filled in with none
missing_categorical_none = ['MasVnrType', 'GarageType', 'Fence', 'MiscFeature']  
housing[missing_categorical_none] = housing[missing_categorical_none].fillna('None')

# electrical nulls is filled in with the most common value
housing.Electrical = housing.Electrical.fillna('SBrkr')

# most null numericals are filled in with 0
missing_numerical = ['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                                'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath',
                                'GarageCars', 'GarageArea']
housing[missing_numerical] = housing[missing_numerical].fillna(0)

# missing lot frontage is filled in with the mean ratio of lot frontage to lot area 
proportion = (housing.LotFrontage/housing.LotArea).mean()
housing.LotFrontage.fillna((housing.LotArea * proportion), inplace=True)

# Pool area is changed to yes/no pool 
f = lambda x: 'yes' if x>0 else 'no'
housing.PoolArea = housing.PoolArea.map(f)
housing = housing.rename(columns={"PoolArea": "PoolYN"})

# Garage year is changed to yes/no garage
housing.GarageYrBlt = housing.GarageYrBlt.fillna(0)
housing = housing.rename(columns={"GarageYrBlt": "GarageYN"})
housing.GarageYN = housing.GarageYN.map(f)

# create a variable for how many bathrooms are in the home 
housing['Bathrooms'] = housing.BsmtFullBath + (.5 *housing.BsmtHalfBath) + housing.FullBath + (.5 *housing.HalfBath)

# drop MiscFeature it has very little information 
housing = housing.drop(['MiscFeature'], axis =1).copy()

# the relationships among SalesPrice and OverallQual, KitchenQual, and ExterQual are exponential
housing.OverallQual = (housing.OverallQual) ** 2
housing.KitchenQual = (housing.KitchenQual)**3
housing.ExterQual = (housing.ExterQual)**3

#drop the one major outlier
housing.drop(309, inplace=True)

# call it df
df = housing.copy()

### Outliers

In [108]:
def find_outliers_IQR(df):

   q1=df.quantile(0.25)
   q3=df.quantile(0.75)
   IQR=q3-q1
   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

   return outliers

# remove all outliers 
df_all = df.copy()
outliers = find_outliers_IQR(df_all.SalePrice)
indexes = list(outliers.index)
df_all.drop(indexes, inplace=True)
df_all.to_csv('df_all.csv')


# remove all non-normal sales 
df_all_normal = df.copy()
index_all_normal = df_all_normal[df_all_normal['SaleCondition'] != 'Normal'].index
df_all_normal.drop(index_all_normal, inplace = True)
df_all_normal.to_csv('df_all_normal.csv')


# remove Family, AdjLand, and Alloca sales 
df_faa = df.copy()
index_faa = df_faa[(df_faa['SaleCondition'] == 'Family') | 
                               (df_faa['SaleCondition'] == 'AdjLand') |
                               (df_faa['SaleCondition'] == 'Alloca')].index
df_faa.drop(index_faa, inplace = True)
df_faa.SaleCondition.value_counts()
df_faa.to_csv('df_faa.csv')


# remove all non-normal sales and outliers within each quality group 
df_normal_quality = df_all_normal.copy()

quality = list((np.array(range(1,11)))**2)
indexes = []

for qual in quality:
    outliers = find_outliers_IQR(df_normal_quality.loc[df_normal_quality.OverallQual == qual].SalePrice)
    indexes.extend(list(outliers.index))
    
df_normal_quality.drop(indexes, inplace=True)
df_normal_quality.to_csv('df_normal_quality.csv')


# remove all outliers with quality group 
df_quality = df.copy()

quality = list((np.array(range(1,11)))**2)
indexes = []

for qual in quality:
    outliers = find_outliers_IQR(df_quality.loc[df_quality.OverallQual == qual].SalePrice)
    indexes.extend(list(outliers.index))
    
df_quality.drop(indexes, inplace=True)
df_quality.to_csv('df_quality.csv')

In [112]:
df_some_quality = df_quality.copy()

index_ = df_some_quality[(df_some_quality['SaleCondition'] == 'Family') | 
                               (df_some_quality['SaleCondition'] == 'AdjLand') |
                               (df_some_quality['SaleCondition'] == 'Alloca')].index
df_some_quality.drop(index_, inplace = True)
df_some_quality.SaleCondition.value_counts()
df_some_quality.to_csv('df_some_quality.csv')

In [113]:
df.to_csv('df.csv')

### wait on these 

In [74]:
# remove all points that are outliers in sales price and living area grouped by quality
df_
quality = list((np.array(range(1,11)))**2)
indexes = []

for qual in quality:
    a= find_outliers_IQR(df_quality.loc[df_quality.OverallQual == qual].SalePrice)
    b = find_outliers_IQR(df_quality.loc[df_quality.OverallQual == qual].GrLivArea)
    a = list(a.index)
    b = list(b.index)
    c = set(a) & set(b)
    indexes.extend(c)
    
indexes

[1435, 533, 2471]

In [ ]:
df_
quality = list((np.array(range(1,11)))**2)
indexes = []

for qual in quality:
    a= find_outliers_IQR(df_quality.loc[df_quality.OverallQual == qual].SalePrice)
    b = find_outliers_IQR(df_quality.loc[df_quality.OverallQual == qual].GrLivArea)
    a = list(a.index)
    b = list(b.index)
    c = set(a) & set(b)
    indexes.extend(c)
    
indexes

In [81]:
# remove all non normal sales then remove any outlier in sales price 

[]
[]
set()
